<a href="https://colab.research.google.com/github/Anisha-bhola/lab/blob/main/MENTAL_HEALTH_DISORDER_(USING_ML)(Jupyter_file).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pandas numpy scikit-learn nltk flask streamlit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 48.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 75.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 5.5 MB/s eta 0:00:00


In [2]:
import pandas as pd
import numpy as np
import pickle
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
import zipfile
import os

In [3]:
# Download NLTK dependencies
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('punkt_tab') # Download the punkt_tab data

# Upload dataset in Colab
from google.colab import files
uploaded = files.upload()

# Extract dataset if zipped
for file_name in uploaded.keys():
    if file_name.endswith(".zip"):
        with zipfile.ZipFile(file_name, 'r') as zip_ref:
            zip_ref.extractall("extracted_data")
        dataset_path = "extracted_data/survey.csv"
    else:
        dataset_path = file_name

# Load dataset
df = pd.read_csv(dataset_path)

# Fill missing values
df.fillna(df.mode().iloc[0], inplace=True)

# Encode categorical variables
label_enc = LabelEncoder()
df['Gender'] = label_enc.fit_transform(df['Gender'])

# Text preprocessing function
def preprocess_text(text):
    if pd.isna(text):
        return ""
    text = text.lower()
    text = re.sub(r'\W', ' ', text)
    words = word_tokenize(text)
    words = [word for word in words if word not in stopwords.words('english')]
    return " ".join(words)

# Use "comments" column as text data if available
if "comments" in df.columns:
    df["cleaned_text"] = df["comments"].apply(preprocess_text)
else:
    df["cleaned_text"] = ""

# TF-IDF for text data
vectorizer = TfidfVectorizer(max_features=500)
X_text = vectorizer.fit_transform(df['cleaned_text']).toarray()

# Select numerical features
numerical_features = ["Age"]
df["Stress_level"] = np.random.randint(1, 10, df.shape[0])  # Simulating stress level
numerical_features.append("Stress_level")

# Scale numerical data
scaler = StandardScaler()
X_numerical = scaler.fit_transform(df[numerical_features])

# Combine features
X = np.hstack((X_numerical, X_text))
y = label_enc.fit_transform(df["treatment"])  # Using "treatment" as the label

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train model
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Evaluate model
y_pred = model.predict(X_test)
print(f"Accuracy: {accuracy_score(y_test, y_pred):.2f}")
print(classification_report(y_test, y_pred))

# Save model and preprocessing objects
with open("/content/mental_health_model.pkl", "wb") as f:
    pickle.dump(model, f)

with open("/content/vectorizer.pkl", "wb") as f:
    pickle.dump(vectorizer, f)

with open("/content/scaler.pkl", "wb") as f:
    pickle.dump(scaler, f)

# Download trained models
from google.colab import files
files.download('/content/mental_health_model.pkl')
files.download('/content/vectorizer.pkl')
files.download('/content/scaler.pkl')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


Saving mental health dataset.zip to mental health dataset.zip
Accuracy: 0.48
              precision    recall  f1-score   support

           0       0.49      0.42      0.45       129
           1       0.47      0.54      0.51       123

    accuracy                           0.48       252
   macro avg       0.48      0.48      0.48       252
weighted avg       0.48      0.48      0.48       252



<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [5]:
import pickle
import numpy as np
import pandas as pd

# Load trained model
model = pickle.load(open("mental_health_model.pkl", "rb"))
vectorizer = pickle.load(open("vectorizer.pkl", "rb"))
scaler = pickle.load(open("scaler.pkl", "rb"))

# User input
age = int(input("Enter your age: "))
stress_level = int(input("On a scale of 1-10, how stressed do you feel? "))
text_input = input("Describe your mental state briefly: ")

# Physical symptoms input
fatigue = int(input("On a scale of 1-10, how fatigued do you feel? "))
headache = int(input("Do you experience frequent headaches? (1 = Yes, 0 = No): "))
sleep_issues = int(input("Do you have trouble sleeping? (1 = Yes, 0 = No): "))
body_pain = int(input("Do you experience unexplained body pain? (1 = Yes, 0 = No): "))
heart_palpitations = int(input("Do you frequently feel heart palpitations? (1 = Yes, 0 = No): "))

# Preprocess text input
text_features = vectorizer.transform([text_input]).toarray()

# Convert numerical features into a DataFrame with column names
numerical_data = pd.DataFrame([[age, stress_level]], columns=["Age", "Stress_level"])

# Scale numerical features using the existing scaler
numerical_features = scaler.transform(numerical_data)

# Combine only trained model features
final_features = np.hstack((numerical_features, text_features))

# Predict mental health issue (0 = No Issue, 1 = Possible Issue)
prediction = model.predict(final_features)[0]

# Advanced mental health status classification with a default advice
# Initialize advice with a default value to avoid the NameError
advice = "Please consult with a medical professional for personalized advice."
if stress_level <= 3 and fatigue <= 3 and headache == 0 and sleep_issues == 0:
    health_status = "✅ Stable Mental Health: You seem to be in a good state. Keep practicing self-care!"
    advice = "Maintain a balanced routine with exercise, sleep, and social interactions. Keep up the great work!"
elif 4 <= stress_level <= 6 and (fatigue >= 4 or headache == 1 or sleep_issues == 1):
    health_status = "⚠️ Mild Stress or Fatigue Detected: Try improving sleep quality and reducing stress."
    advice = "Consider deep breathing exercises, maintaining a consistent sleep schedule, and engaging in hobbies to reduce stress."
elif 7 <= stress_level <= 8 or fatigue >= 5:
    health_status = "🟠 Moderate Mental Health & Physical Concerns: Persistent stress and physical symptoms detected."
    advice = "Incorporate relaxation techniques like meditation or yoga. Stay hydrated and maintain a nutritious diet. Talking to a trusted friend or professional may help."
elif stress_level >= 9 or (fatigue >= 7 and sleep_issues == 1 and heart_palpitations == 1):
    health_status = "🚨 Severe Mental & Physical Risk: High stress and multiple health issues detected."
    advice = "It is highly recommended to consult a mental health professional. Prioritize rest, avoid caffeine, and seek immediate support from family or medical professionals."

# Display results
print("\n🔍 **Mental & Physical Health Assessment** 🔍")
print(f"🧑 Age: {age}")
print(f"📊 Stress Level: {stress_level}/10")
print(f"💬 Mental State Description: {text_input}")
print(f"😴 Fatigue Level: {fatigue}/10")
print(f"🤕 Headaches: {'Yes' if headache else 'No'}")
print(f"🌙 Sleep Issues: {'Yes' if sleep_issues else 'No'}")
print(f"💢 Body Pain: {'Yes' if body_pain else 'No'}")
print(f"💓 Heart Palpitations: {'Yes' if heart_palpitations else 'No'}")
print("\n🔹 **Health Status:**", health_status)
print(f"💡 **Advice:** {advice}")

Enter your age: 21
On a scale of 1-10, how stressed do you feel? 5
Describe your mental state briefly: average
On a scale of 1-10, how fatigued do you feel? 8
Do you experience frequent headaches? (1 = Yes, 0 = No): 1
Do you have trouble sleeping? (1 = Yes, 0 = No): 1
Do you experience unexplained body pain? (1 = Yes, 0 = No): 1
Do you frequently feel heart palpitations? (1 = Yes, 0 = No): 0

🔍 **Mental & Physical Health Assessment** 🔍
🧑 Age: 21
📊 Stress Level: 5/10
💬 Mental State Description: average
😴 Fatigue Level: 8/10
🤕 Headaches: Yes
🌙 Sleep Issues: Yes
💢 Body Pain: Yes
💓 Heart Palpitations: No

🔹 **Health Status:** ⚠️ Mild Stress or Fatigue Detected: Try improving sleep quality and reducing stress.
💡 **Advice:** Consider deep breathing exercises, maintaining a consistent sleep schedule, and engaging in hobbies to reduce stress.
